In [35]:
import pandas as pd
import numpy as np

In [36]:
import matplotlib.pyplot as plt

In [37]:
from datetime import datetime

In [38]:
#Current SQL query
"""
SELECT 
geo__country, 
geo__city,
event_params__video_title,
event_params__videotitle,
event_name,
COUNT(event_name) AS times_completed,
FROM `jfp-data-warehouse.production_firebase.firebase_event` 
WHERE geo__country in ('Russia', 'China', 'Turkey', 'Thailand', 'Iran', 'Vietnam', 'Egypt', 'Japan', 'Bangladesh', 'Pakistan', 'Indonesia', 'India') AND 
event_params__video_title IS NOT NULL AND
event_name IN ('videostarts',  'videoplay',  'a_media_progress10',  'a_media_progress25',  'a_media_progress50',  'a_media_progress75',  'a_media_progress90') AND 
DATE(event_date_partition) 
BETWEEN  "2021-01-01" AND "2021-03-31" 
GROUP BY 
geo__country, 
geo__city,
event_params__videotitle, 
event_params__video_title,
event_name
ORDER BY 
geo__country,
event_params__video_title,
times_completed DESC 
LIMIT 15000
""";

In [39]:
data = pd.read_csv('/Users/djbetts/Desktop/jf_project/data/recommender_base.csv')

In [40]:
df = data.copy()
df.drop(['geo__city', 'times_completed'], axis=1, inplace=True)

In [41]:
df.columns

Index(['geo__country', 'event_params__video_title', 'event_params__videotitle',
       'event_name'],
      dtype='object')

In [42]:
df['video_title'] = df['event_params__videotitle'].fillna(df['event_params__video_title'])

In [43]:
df.drop(['event_params__videotitle','event_params__video_title'], axis=1, inplace=True)

In [44]:
(df.geo__country.value_counts())

India         8644
Egypt         2467
Bangladesh    1953
China         1936
Name: geo__country, dtype: int64

In [45]:
df.event_name.value_counts()

videoplay             2741
videostarts           2670
a_media_progress10    2230
a_media_progress25    2093
a_media_progress50    1908
a_media_progress75    1731
a_media_progress90    1627
Name: event_name, dtype: int64

In [46]:
df.video_title.value_counts()

JESUS                         2988
Childhood of Jesus             893
Baptism of Jesus by John       876
Birth of Jesus                 856
Abraham                        345
                              ... 
奮鬥與成就                            1
使徒行傳                             1
Jesus Taken Up Into Heaven       1
This Is Freedom                  1
#फालिंगप्लेट्स                   1
Name: video_title, Length: 406, dtype: int64

In [47]:
df_90 = df[df['event_name'] == 'a_media_progress90']

In [48]:
df_90.video_title.unique()

array(['#FallingPlates', '1. Jesus, Our Loving Pursuer',
       '7. Jesus Our Living Water', 'Angels at the Tomb', 'B Me',
       'Baptism of Jesus by John', 'Beatitudes', 'Birth of Jesus',
       'Blessed are those Who Hear and Obey', 'Breathe',
       'Burial of Jesus', 'Can the Bible Be Trusted?',
       'Childhood of Jesus', 'Creation', 'Crucified Convicts',
       'Death of Jesus', 'Delight', 'Disciples Chosen', 'Doll Face',
       'God&#39;s Rescue Plan', 'Great Commission and Ascension',
       'Healing of the Demoniac', 'Invisible', 'JESUS',
       'Jairus&#39;s Daughter Brought Back to Life',
       'Jesus Calms the Storm', 'Jesus Carries His Cross',
       'Jesus Feeds 5,000', 'Jesus Predicts His Death and Resurrection',
       'Jesus Proclaims Fulfillment of the Scriptures',
       'Jesus Spends Time with Sinners', 'Jesus is Betrayed and Arrested',
       'Jesus is Brought to Herod', 'Jesus is Crucified',
       'Jesus is Mocked and Questioned', 'Jesus is Sentenced',
       

In [49]:
df_90.geo__country.value_counts()

India         937
Egypt         287
Bangladesh    207
China         196
Name: geo__country, dtype: int64

In [50]:
country_groupby = df.groupby('geo__country')

In [51]:
country_groupby['video_title'].unique()

geo__country
Bangladesh    [#FallingPlates, 1. Jesus, Our Loving Pursuer,...
China         [#FallingPlates, #掉落的盘子, 40天与耶稣同在, 7天与耶稣同在：恩典,...
Egypt         [#FallingPlates, 1- في العائلة, 2- عمل يدوي, 2...
India         [#FallingPlates, #फालिंगप्लेट्स , 1. Jesus, Ou...
Name: video_title, dtype: object

In [52]:
india = country_groupby.get_group('India')

In [53]:
falling_plates_india = india[india['video_title'] == '#FallingPlates']

In [54]:
event_score = falling_plates_india.event_name.value_counts()

In [55]:
event_score.index

Index(['videostarts', 'videoplay', 'a_media_progress10', 'a_media_progress25',
       'a_media_progress50', 'a_media_progress75', 'a_media_progress90'],
      dtype='object')

In [56]:
keys = df.video_title.unique()
reviews = dict.fromkeys(keys, 0)

In [57]:
for row in df.columns:
    print(row)

geo__country
event_name
video_title


In [25]:
#tried to make a function by iterating through a groupby. couldn't figure it out.

# def rating_calculator(group):
#     rating = 0
#     reviews = dict.fromkeys(set(group.video_title), 0)
#     for event_name in group.event_name:
#         if event_name == 'videostarts' or event_name == 'videoplay':
#             reviews[group.video_title] += 1
#         elif event_name == 'a_media_progress10':
#             reviews[group.video_title] += (1 * .1)
#         elif event_name == 'a_media_progress25':
#             reviews[group.video_title] += (1 * .25)
#         elif event_name == 'a_media_progress50':
#             reviews[group.video_title] += (1 * .5)
#         elif event_name == 'a_media_progress75':
#             reviews[group.video_title] += (1 * .75)
#         elif event_name == 'a_media_progress90':
#             reviews[group.video_title] += (1 * .9)
#     print(f'total: {rating}\nnormalized: {rating/len(events)}')
#     return (reviews)
# #     return
# # print(rating)
# # print(rating/len(event_score))

In [26]:
#rating_calculator(event_score)

In [27]:
def review_builder(country_groupby):
    review_dic = dict.fromkeys(country_groupby.groups.keys(), None)
    print(review_dic)
    for country, group in country_groupby:
        review_dic[country] = rating_calculator(group)
    return review_dic

In [28]:
(29 + 28 + (22*.1) + (21*.25) + (20*.5) + (13*.9) + (13*.75))

95.9

In [32]:
def view_to_rating(event_name):
    d ={
        'videostarts':1,
        'videoplay':1, 
        'a_media_progress10':(1*.1),
        'a_media_progress25':(1*.25),
        'a_media_progress50':(1*.5), 
        'a_media_progress75':(1*.75), 
        'a_media_progress90':(1*.9)
    }
    
    return(d[event_name])

In [34]:
view_to_rating('a_media_progress90')

0.9

In [58]:
df['score'] = df.event_name.apply(view_to_rating)

In [83]:
temp = df.drop(['event_name'], axis=1)

In [84]:
pivot = pd.pivot_table(data=temp, index=['geo__country'])

In [85]:
pivot

,score
geo__country,
Bangladesh,0.645008
China,0.666942
Egypt,0.652047
India,0.661071


In [63]:
groupby = df.groupby(['geo__country','video_title'])

In [67]:
review_scores = groupby.sum()

In [82]:
review_scores.index

MultiIndex([('Bangladesh',                        '#FallingPlates'),
            ('Bangladesh',          '1. Jesus, Our Loving Pursuer'),
            ('Bangladesh',       '2. Jesus, Our Gracious Forgiver'),
            ('Bangladesh',       '6. Jesus, Our Complete Restorer'),
            ('Bangladesh',             '7. Jesus Our Living Water'),
            ('Bangladesh',                               'Abraham'),
            ('Bangladesh',                    'Angels at the Tomb'),
            ('Bangladesh', 'Annas Questions Jesus&#39;s Authority'),
            ('Bangladesh',                  'Announcement to Mary'),
            ('Bangladesh',                                  'B Me'),
            ...
            (     'India',          'How to Know Jesus Personally'),
            (     'India',                       'Idols in Athens'),
            (     'India',                               'In Time'),
            (     'India',                         'In the Family'),
            (     

In [402]:
country_groupby.size()

geo__country
Bangladesh    1953
China         1936
Egypt         2467
India         8644
dtype: int64

In [72]:
country_groupby.describe()

times_completed                                             \
                       count       mean         std  min  25%  50%  75%   
geo__country                                                              
Bangladesh            1953.0  14.890425  168.788664  1.0  1.0  1.0  3.0   
China                 1936.0   1.858471    2.846569  1.0  1.0  1.0  2.0   
Egypt                 2467.0   8.665180   51.231910  1.0  1.0  1.0  3.0   
India                 8644.0  29.642411  289.741494  1.0  1.0  2.0  5.0   

                      
                 max  
geo__country          
Bangladesh    5096.0  
China           39.0  
Egypt         1355.0  
India         9308.0

In [354]:
index = df.geo__country.unique()

In [355]:
columns = df.video_title.unique()

In [390]:
data = np.zeros((index.shape[0], columns.shape[0]))

In [367]:
df.event_name.value_counts().index

Index(['videoplay', 'videostarts', 'a_media_progress10', 'a_media_progress25',
       'a_media_progress50', 'a_media_progress75', 'a_media_progress90'],
      dtype='object')

In [395]:
reviews = pd.DataFrame(data, index=index, columns=columns)

In [396]:
reviews

,#FallingPlates,"1. Jesus, Our Loving Pursuer","2. Jesus, Our Gracious Forgiver","6. Jesus, Our Complete Restorer",7. Jesus Our Living Water,Abraham,Angels at the Tomb,Annas Questions Jesus&#39;s Authority,Announcement to Mary,B Me,...,Fracture,Getting Started Is Easy,God&#39;s Word Becomes Flesh,Good,Great Confrontation,Grief will Turn to Joy!,Growing In Christ,Handiwork,Idols in Athens,In Time
Bangladesh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
China,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egypt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
India,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [397]:
df.head()

,geo__country,event_name,video_title
0,Bangladesh,videoplay,#FallingPlates
1,Bangladesh,a_media_progress10,#FallingPlates
2,Bangladesh,a_media_progress90,#FallingPlates
3,Bangladesh,videostarts,#FallingPlates
4,Bangladesh,a_media_progress75,#FallingPlates
